In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime

# Updating data from the CSV File published by John Hopkins University

ipython magic commands avoided here, because at the end, this notebook is used for generated a python code

In [4]:
updateRawCmd = 'sh ../data_raw/updateRaw.sh' 
os.system(updateRawCmd)

0

In [5]:
df_raw = pd.read_csv('../data_raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')

In [6]:
df_raw.head(12)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,37345,37424,37431,37551,37596,37599,37599,37599,37856,37894
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,6817,6971,7117,7260,7380,7499,7654,7812,7967,8119
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,36699,37187,37664,38133,38583,39025,39444,39847,40258,40667
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,977,981,989,989,989,1005,1005,1024,1024,1045
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,1762,1815,1852,1879,1906,1935,1966,2015,2044,2068
5,NaN,Antigua and Barbuda,17.06080,-61.796400,0,0,0,0,0,0,...,92,92,93,93,93,93,93,94,94,94
6,NaN,Argentina,-38.41610,-63.616700,0,0,0,0,0,0,...,268574,276072,282437,289100,294569,299126,305966,312659,320884,329043
7,NaN,Armenia,40.06910,45.038200,0,0,0,0,0,0,...,40794,41023,41299,41495,41663,41701,41846,42056,42319,42477
8,Australian Capital Territory,Australia,-35.47350,149.012400,0,0,0,0,0,0,...,113,113,113,113,113,113,113,113,113,113
9,New South Wales,Australia,-33.86880,151.209300,0,0,0,0,3,4,...,3927,3936,3945,3950,3957,3959,3966,3971,3972,3981


# Processing Data

the desired data format: 

a table (panda dataframe) which shows the 
number of cumulative confirmed cases for *each* **country** and **date**. 

In particular 
* for each Country/Region, compute the sum of confirmed cases as a time series (among its Provience/State)
* reformat the dates into ISO format: yyyy-mm-dd
* rename Country/Region as country

which looks like:

| date (as index) | Afghanistan | Albania | ... |
| --------------- | ----------- | ------- | --- |
| 2020-01-22      | ...         | ...     | ... | 
| 2020-01-22      | ...         | ...     | ... | 
| ...             | ...         | ...     | ... | 
| (Today)         | ...         | ...     | ... | 


## create the dataframe & formating the dates

In [ ]:
dateList =[datetime.strptime( eachOldStr,"%m/%d/%y") for eachOldStr in df_raw.columns[4:]] 
dateStrNew = [eachDate.strftime('%Y-%m-%d') for eachDate in dateList]
df_proc = pd.DataFrame({'date':dateStrNew})
df_proc.set_index('date',inplace=True)
del dateList
del dateStrNew

## add a column for every country to the new dataframe

In [ ]:
countries = df_raw['Country/Region'].unique() # returning a list
# alternative (as long as I won't use integer indexing for the list above): countries = set(df_raw['Country/Region'])  
# print(countries)

In [ ]:
# to inform yourself how many countries are involved, run:
# len(countries)

In [ ]:
for country in countries:
    row_filter = df_raw['Country/Region']==country
    df_proc[country]=np.array(df_raw[row_filter].loc[:,'1/22/20':].sum(axis=0))

the following is just a preview for df_proc

In [ ]:
df_proc.tail(10)  # suppressed for export

# Outputing the (Processed) Data as CSV

In [ ]:
%matplotlib inline
# df_proc['Australia'].plot()

In [ ]:
df_proc.to_csv('../data_proc/COVID_TimeSeries_by_country.csv',sep=';')